In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
!pip install tenacity

In [ ]:
from google.colab import drive
from google.colab import userdata
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
from openai import OpenAI
from tenacity import retry, stop_after_attempt, wait_fixed
from tqdm import notebook
import pandas as pd
import numpy as np
import time
import random
import copy
import json

base_dir = '/content/drive/MyDrive/SemEval/'
train_dir = base_dir + 'train_data.csv'
# val_dir = base_dir + 'val_data.csv'
val_dir = base_dir + 'new_test_data_nolabel.csv'

In [5]:
template = '''
Which option is the answer to this riddle, explain in a step by step manner:
<RIDDLE>
1) <OPTION1>
2) <OPTION2>
3) <OPTION3>
4) None of the above.
please place your answer in a json format:
{
  "option_number": <JUST_THE_NUMBER_OF_THE_CORRECT_OPTION>,
  "explanation": <EXPLANATION_WHY_IT_IS_CORRECT>
}'''

In [6]:
def gen_prompt(template, sample):
  prompt = copy.deepcopy(template)
  prompt = prompt.replace('<RIDDLE>', sample['question'])
  prompt = prompt.replace('<OPTION1>', sample['option1'])
  prompt = prompt.replace('<OPTION2>', sample['option2'])
  prompt = prompt.replace('<OPTION3>', sample['option3'])
  return prompt

In [13]:
train_df = pd.read_csv(train_dir)
val_df = pd.read_csv(val_dir)
print(train_df.columns, '\t', train_df.shape)
print(val_df.columns, '\t', val_df.shape)

Index(['Unnamed: 0', 'id', 'question', 'answer', 'distractor1', 'distractor2',
       'label', 'q_token_count', 'a_token_count', 'd1_token_count',
       'd2_token_count'],
      dtype='object') 	 (507, 11)
Index(['Unnamed: 0', 'question', 'option1', 'option2', 'option3', 'option4'], dtype='object') 	 (120, 6)


In [14]:
samples = train_df.sample(n=2)
gen_prompt(template, val_df.iloc[0])

'\nWhich option is the answer to this riddle, explain in a step by step manner:\nIn a small village, two farmers are working in their fields - a diligent farmer and a lazy farmer. The hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Can you explain this unusual relationship?\n1) The lazy farmer is his mother.\n2) The lazy farmer is not a responsible father as he is lazy.\n3) The diligent farmer devoted himself to the farm and gradually forgot his father.\n4) None of the above.\nplease place your answer in a json format:\n{\n  "option_number": <JUST_THE_NUMBER_OF_THE_CORRECT_OPTION>,\n  "explanation": <EXPLANATION_WHY_IT_IS_CORRECT>\n}'

In [15]:
@retry(stop=stop_after_attempt(3), wait=wait_fixed(25))
def gpt_inference(client, prompt):
  messages = []
  messages.append(
      {"role": "system", "content": "You are a reasonable and accurate assistant"}
  )
  messages.append(
      {"role": "user", "content": prompt},
  )

  chat = client.chat.completions.create(
        messages=messages,
        model="gpt-3.5-turbo",
        temperature = 0.0
    )
  response = chat.choices[0].message.content
  return response

In [16]:
def extract_ans(json_ans, sample):
  dict_ans = json.loads(json_ans)
  if dict_ans['option_number'] == 1:
    prediction = sample['option1']
  elif dict_ans['option_number'] == 2:
    prediction = sample['option2']
  elif dict_ans['option_number'] == 3:
    prediction = sample['option3']
  else:
    prediction = 'None of the above.'
  return prediction, dict_ans['explanation']

In [17]:
json_string = """
{
  "option_number": 1,
  "explanation": "The lazy farmer is his mother. This is because the riddle states that the hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Therefore, the only logical conclusion is that the lazy farmer is the mother of the hardworking farmer."
}
"""
extract_ans(json_string, val_df.iloc[0])

('The lazy farmer is his mother.',
 'The lazy farmer is his mother. This is because the riddle states that the hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Therefore, the only logical conclusion is that the lazy farmer is the mother of the hardworking farmer.')

In [20]:
def run(val_df):
  client = OpenAI(
    api_key = userdata.get('GPT_API_KEY')
  )
  pred_df = copy.deepcopy(val_df)
  for i, sample in notebook.tqdm(pred_df.iterrows(), total=pred_df.shape[0]):
    prompt = gen_prompt(template, sample)
    json_answer = gpt_inference(client, prompt)
    prediction, explanation = extract_ans(json_answer, sample)
    pred_df.loc[i, 'gpt_prediction'] = prediction
    pred_df.loc[i, 'gpt_explanation'] = explanation
    # pred_df.loc[i, 'is_true'] = (sample['answer'] == prediction)
    time.sleep(20)
  return pred_df

In [21]:
new_val_df = run(val_df)
new_val_df.head(10)

  0%|          | 0/120 [00:00<?, ?it/s]

,Unnamed: 0,question,option1,option2,option3,option4,gpt_prediction,gpt_explanation
0,0,"In a small village, two farmers are working in...",The lazy farmer is his mother.,The lazy farmer is not a responsible father as...,The diligent farmer devoted himself to the far...,None of above.,The lazy farmer is his mother.,The lazy farmer is his mother. This explains t...
1,1,Romeo and Juliet are discovered dead on the be...,They were sleeping and scared by the sound of ...,The rumble of the train moved the shelf which ...,Romeo and Juliet are fish. The rumble of the t...,None of above.,Romeo and Juliet are fish. The rumble of the t...,Option 3 is the correct answer. Romeo and Juli...
2,2,How many years in your life it happens that be...,In ech leap year.,In the first year of graduation.,It happens every year.,None of above.,It happens every year.,The correct option is 3) It happens every year...
3,3,Who would serve as the team's captain if a cru...,The first officer.,The captain.,The second officer.,None of above.,The captain.,The captain would serve as the team's captain ...
4,4,"In one city, 5% of the population has an unlis...",One hundred people.,Ninty-five people.,Five people.,None of above.,Five people.,If 5% of the population has an unlisted phone ...
5,5,Two entries were on duty outside a barracks. O...,Although the guards were looking in opposite d...,One of the guards had an unusual vision so he ...,The smiling guard's action is such big that ev...,None of above.,Although the guards were looking in opposite d...,The correct option is 1) Although the guards w...
6,6,A man wagered $50 on his dog's ability to jump...,A house can not jump.,That dog species were famous for their jumping...,The height of the tiny house is lower than the...,None of above.,A house can not jump.,The correct option is 1) A house can not jump....
7,7,How many times can you divide 100 by 10?,Infinite time.,Once.,Two times.,None of above.,Once.,The correct option is 2) Once. When you divide...
8,8,"In a city, there are so many criminals that po...",Twenty people.,Eighty people.,Two hundred people.,None of above.,Eighty people.,If the number of criminals in the city is appr...
9,9,When did Christmas and New Year's Day fall on ...,In the period of BC.,It happens every year.,In the first year of each century.,None of above.,None of the above.,None of the above. Christmas and New Year's Da...


In [24]:
test_dir = base_dir + 'new_test_data_nolabel.csv'
test_df = pd.read_csv(test_dir)

def extract_id(df):
  for i, row in df.iterrows():
    question = row['question']
    option1 = row['option1']
    option2 = row['option2']
    option3 = row['option3']
    prediction = row['gpt_prediction']
    if option1 == prediction:
      id = 0
    elif option2 == prediction:
      id = 1
    elif option3 == prediction:
      id = 2
    else:
      id = 3
    df.loc[i, 'pred_id'] = int(id)

  return df

def write_answer_id(df, path):
  with open(path, 'w') as f:
    for i, row in df.iterrows():
      f.write(str(int(row['pred_id']))+'\n')

In [25]:
new_val_df = extract_id(new_val_df)
new_val_df

,Unnamed: 0,question,option1,option2,option3,option4,gpt_prediction,gpt_explanation,pred_id
0,0,"In a small village, two farmers are working in...",The lazy farmer is his mother.,The lazy farmer is not a responsible father as...,The diligent farmer devoted himself to the far...,None of above.,The lazy farmer is his mother.,The lazy farmer is his mother. This explains t...,0.0
1,1,Romeo and Juliet are discovered dead on the be...,They were sleeping and scared by the sound of ...,The rumble of the train moved the shelf which ...,Romeo and Juliet are fish. The rumble of the t...,None of above.,Romeo and Juliet are fish. The rumble of the t...,Option 3 is the correct answer. Romeo and Juli...,2.0
2,2,How many years in your life it happens that be...,In ech leap year.,In the first year of graduation.,It happens every year.,None of above.,It happens every year.,The correct option is 3) It happens every year...,2.0
3,3,Who would serve as the team's captain if a cru...,The first officer.,The captain.,The second officer.,None of above.,The captain.,The captain would serve as the team's captain ...,1.0
4,4,"In one city, 5% of the population has an unlis...",One hundred people.,Ninty-five people.,Five people.,None of above.,Five people.,If 5% of the population has an unlisted phone ...,2.0
...,...,...,...,...,...,...,...,...,...
115,115,A professional fisherman caught 30 fish during...,Two.,One.,Three.,None of above.,Three.,The fisherman caught 30 fish in total over 5 d...,2.0
116,116,Bob was working on a project when suddenly int...,He needed a membership to search in google.,He searched the google in a wrong way.,The internet was disconnected so he couldn't s...,None of above.,The internet was disconnected so he couldn't s...,The internet was disconnected so he couldn't s...,2.0
117,117,"He has wed numerous women, but never himself. ...",A teacher.,A preacher.,A laywer.,None of above.,A preacher.,The correct option is 2) A preacher. The riddl...,1.0
118,118,You walk into a room and see a bed and lie on ...,"Thirty-six, as there are eighteen animals.","Six. The bed's four legs, plus your two legs.","Seventy-two, as there are eighteen animals.",None of above.,"Six. The bed's four legs, plus your two legs.",The correct answer is option 2. The riddle sta...,1.0


In [27]:
path = base_dir + 'answer_sen.txt'
write_answer_id(new_val_df, path)

In [ ]:
new_val_df['is_true'].sum() / new_val_df.shape[0] * 100

66.66666666666666

In [ ]:
target_path = base_dir + 'new_val_df.csv'
new_val_df.to_csv(target_path)